<h1>Project: Personal Assistant F.R.I.D.A.Y</h1>

<h3>Loading necessary libraries</h3>

In [ ]:
import speech_recognition as sr 
import os 
import webbrowser
import datetime
import openai
import subprocess
import requests


<h3>OpenAI API key</h3>

In [ ]:
openai.api_key = 'put-your-api-key-here'

<h3>Command Function to Interpret the Query</h3>

In [ ]:
def command():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        r.pause_threshold = 1
        print("Listening... ")
        audio = r.listen(source)
        try:
            query = r.recognize_google(audio, language = "en-in")
            print(f"User said: {query}")
            return query 
        except sr.UnknownValueError:
            say("Sorry, I did not understand that.")
            return ""
        except sr.RequestError:
            say("Sorry, my speech service is down.")
            return ""
        except Exception as e:
            say("Sorry, there was an error.")
            return ""

<h3>'Say' function to print the 'query' asked by the user</h3>

In [ ]:
def say(text):
    print(f"DEBUG: Saying: {text}")
    os.system(f"say '{text}'")

<h3>'Add task' function to store the tasks</h3>

In [ ]:
tasks = []

def add_task(task):
    tasks.append(task)
    say(f"Task '{task}' added.")

def view_tasks():
    if not tasks:
        say("No tasks available.")
    else:
        say("Here are your tasks:")
        for i, task in enumerate(tasks, start=1):
            say(f"Task {i}: {task}")

def delete_task(command):
    try:
        # Extract task number from the command
        task_number = int(command.split()[-1])
        if 1 <= task_number <= len(tasks):
            removed_task = tasks.pop(task_number - 1)
            say(f"Task '{removed_task}' deleted.")
        else:
            say("Invalid task number.")
    except ValueError:
        say("Please provide a valid task number.")
    except IndexError:
        say("Please provide the task number.")

<h3>'get_weather' function to obtain the weather information of any location across the globe</h3>

In [ ]:
def get_weather(location):
    api_key = "put-your-api-key-here"
    weather_url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}"
    response = requests.get(weather_url)
    weather_data = response.json()
    if weather_data["cod"] == 200:
        main = weather_data["main"]
        wind = weather_data["wind"]
        weather_description = weather_data["weather"][0]["description"]
        temperature = main["temp"]
        say(f"The weather in {location} is {weather_description} with a temperature of {temperature - 273.15:.2f} degrees Celsius.")
    else:
        say(f"Sorry, I couldn't fetch the weather information for {location}.")

<h3>'get_news' function to get the Top 5 headlines across the globe</h3>

In [ ]:
def get_news():
    api_key = "put-your-api-key-here"
    news_url = f"http://newsapi.org/v2/top-headlines?country=us&apiKey={api_key}"
    response = requests.get(news_url)
    news_data = response.json()
    if news_data["status"] == "ok":
        top_headlines = news_data["articles"][:5]  # Get top 5 headlines
        for i, article in enumerate(top_headlines):
            say(f"Headline {i+1}: {article['title']}")
    else:
        say("Sorry, I couldn't fetch the news.")

<h3>Site closing function</h3>

In [ ]:
def close_site(site_name):  # to do, this is still not functioning properly
    print(f"DEBUG: Site name: {site_name}")
    if "chrome" in site_name:
        browser = "Google Chrome"
    elif "firefox" in site_name:
        browser = "Firefox"
    elif "safari" in site_name:
        browser = "Safari"
    else:
        say("Site closing not supported for this browser.")
        return

    applescript = f'tell application "{browser}" to close (every tab of window 1)'
    subprocess.run(['osascript', '-e', applescript], shell=True)
    say(f"{browser} tabs closed.")

In [ ]:
def get_openai_response(prompt):
    try:
        response = openai.Completion.create(
            engine='text-davinci',  # Use the desired engine here
            prompt=prompt,
            max_tokens=150
        )
        return response.choices[0].text.strip()
    except Exception as e:
        print(f"Error with OpenAI API: {e}")
        return "Sorry, I couldn't process that."

<h3>Greeting Message</h3>

In [ ]:
say("Hello, I am Friday! Your personal assistant!")

<h3>Actual Functional Loop</h3>

In [ ]:
while True:
    query = command().lower()
    if not query:
        continue  # Skip the rest of the loop if there was an error or unrecognized speech

    if "terminate yourself" in query:
        say("Goodbye sir, Please press the run button to call me again, Have a nice day!")
        break


    if "activate advanced version" in query:
        say("Advanced version activated. How can I assist you further?")
        while True:
            query = command().lower()
            if "stop" in query:
                say("Getting out of open.A.I.")
                break
            response = get_openai_response(query)
            say(response)
        continue  # After stopping advanced mode, continue to the main loop

    if "weather update" in query:
        say("Please specify the location.")
        location_query = command().lower()
        if location_query:
            get_weather(location_query)
        else:
            say("Sorry, I didn't get the location.")
    
    if "news update" in query:
        get_news()

    if "add a task" in query:
        say("What task would you like to add?")
        task = command().lower()
        if task:
            add_task(task)
        else:
            say("Sorry, I didn't get the task.")

    if "view tasks" in query:
        view_tasks()

    if "delete a task" in query:
        say("Which task number would you like to delete?")
        task_number = command().lower()
        if task_number:
            delete_task(task_number)
        else:
            say("Sorry, I didn't get the task number.")




#todo: Add more sites
    sites = [["google", "https://www.google.com"], ["youtube", "https://www.youtube.com"], 
             ["wikipedia", "https://www.wikipedia.com"], ["amazon", "https://www.amazon.com"], ["instagram", "https://www.instagram.com"], ["facebook", "https://www.facebook.com"], ["leetcode", "https://www.leetcode.com"], ["coursera", "https://www.coursera.com"], ["udemy", "https://www.udemy.com"], ["codechef", "https://www.codechef.com"],["codeforces", "https://www.codeforces.com"], ["linkedin", "https://www.linkedin.com"]] ## opens via default browser.
    
    for site in sites:  # Loop through the list of sites
        if f"open {site[0]}" in query:
            say(f"Opening {site[0]}, sir")   
            webbrowser.open(site[1])
            break
        elif f"close {site[0]}" in query:
            close_site((site[0]).lower())
            break

    if "open music" in query:
        musicpath = "/Users/atharvakulkarni/Downloads/track.mp3"
        os.system(f"open {musicpath}")
    
    if "the time" in query:
        now = datetime.datetime.now()
        say(f"Sir, the time is {now.hour} hours {now.minute} minutes and {now.second} seconds")

    if "open notion" in query:
        notion = "/Applications/Notion.app"
        os.system(f"open {notion}")

    if "open facetime".lower() in query.lower(): 
        facetime = "/Application/Facetime.app"
        os.system(f"open {facetime}")

    if "open firefox" in query:
        firefox = "/Applications/Firefox.app"
        os.system(f"open {firefox}")

    if "open app store" in query:
        app_store = "/Applications/App Store.app"
        os.system(f"open {app_store}") ##problem in opening this as of now   

    if "open spotify" in query:
        spotify = "/Applications/Spotify.app"
        os.system(f"open {spotify}")

    if "open whatsapp" in query:
        whatsapp = "/Applications/WhatsApp.app"
        os.system(f"open {whatsapp}")

    if "open terminal" in query:
        terminal = "/System/Applications/Utilities/Terminal.app"
        os.system(f"open {terminal}")

    if "open safari" in query:
        safari = "/Applications/Safari.app"
        os.system(f"open {safari}")  

    if "close notion" in query:
        os.system("pkill Notion")
        say("Notion closed")

    if "close facetime" in query:
        os.system("pkill FaceTime")
        say("FaceTime closed")

    if "close firefox" in query:
        os.system("pkill Firefox")
        say("Firefox closed")

    if "close app store" in query:
        os.system("pkill 'App Store'")
        say("App Store closed")

    if "close spotify" in query:
        os.system("pkill Spotify")
        say("Spotify closed")

    if "close whatsapp" in query:
        os.system("pkill WhatsApp")
        say("WhatsApp closed")

    if "close terminal" in query:
        os.system("pkill Terminal")
        say("Terminal closed")

    if "close safari" in query:
        os.system("pkill Safari")
        say("Safari closed")